<a href="https://colab.research.google.com/github/gongboogi/Python-Practice/blob/main/Pandas%EB%A1%9C%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%8B%A4%EB%A3%A8%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 판다스를 이용해 데이터 다루기

In [3]:
# 라이브러리 임포트
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

# 시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 소수점 세 번째 자리까지 표시
%precision 3

'%.3f'

# 판다스로 데이터를 다루는 기본적인 방법

## 1. 계층적 인덱스

데이터를 여러 개의 변수를 기준으로 집계할 때 계층적 인덱스가 편리하다. 다음 코드는 데이터 세트를 2층 인덱스 구조로 설정했다.

In [4]:
# 3열 3행 데이터를 생성하고 인덱스와 칼럼명 설정
hier_df = DataFrame(
    np.arange(9).reshape((3,3)),
    index = [
        ['a', 'a', 'b'],
        [1,2,2]
    ],

    columns = [
        ['Pusan', 'Seoul', 'Pusan'],
        ['Blue', 'Red', 'Red']
    ]
)

hier_df

Pusan Seoul Pusan
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [5]:
# index에 이름 붙이기: names
hier_df.index.names = ['key1', 'key2']

# 칼럼에 이름 붙이기: names
hier_df.columns.names = ['city', 'color']
hier_df

city      Pusan Seoul Pusan
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

In [7]:
# 칼럼 범위 축소: city가 Pusan인 데이터만 보기
hier_df['Pusan']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

**인덱스 기준 집계**

In [8]:
# 계층별 요약통계량: 행 합계
hier_df.sum(level = 'key2', axis = 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


city,Pusan,Seoul,Pusan
color,Blue,Red,Red
key2,,,
1,0,1,2
2,9,11,13


In [9]:
# 열 합계: axis=1
hier_df.sum(level = 'color', axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15

**인덱스 원소 삭제**

특정 인덱스를 삭제할 때는 drop 메서드를 사용한다.

In [10]:
#key1의 b 삭제
hier_df.drop(['b'])

city      Pusan Seoul Pusan
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5



---



##연습문제

1. 다음 데이터에서 Daegu 열만 추출하세요.


In [11]:
hier_df1 = DataFrame(
  np.arange(12).reshape((3,4)),
  index = [['c', 'd', 'd'], [1,2,1]],
  columns = [
    ['Daegu', 'Daejeon', 'Gangeung', 'Daegu'],
    ['Yellow', 'Yellow', 'Red', 'Blue']
  ]
)

hier_df1.index.names = ['key1', 'key2']
hier_df1.columns.names = ['city', 'color']
hier_df1

city       Daegu Daejeon Gangeung Daegu
color     Yellow  Yellow      Red  Blue
key1 key2                              
c    1         0       1        2     3
d    2         4       5        6     7
     1         8       9       10    11

In [16]:
hier_df1['Daegu']

color      Yellow  Blue
key1 key2              
c    1          0     3
d    2          4     7
     1          8    11

2. 1의 데이터에서 city별 평균값을 산출하세요.

In [21]:
hier_df1.mean(level = 'city', axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  """Entry point for launching an IPython kernel.


city       Daegu  Daejeon  Gangeung
key1 key2                          
c    1       1.5      1.0       2.0
d    2       5.5      5.0       6.0
     1       9.5      9.0      10.0

3. 1의 데이터에서 key2별로 각 행의 합계를 산출하세요.

In [25]:
hier_df1.sum(level = 'key2')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


city,Daegu,Daejeon,Gangeung,Daegu
color,Yellow,Yellow,Red,Blue
key2,,,,
1,8,10,12,14
2,4,5,6,7




---



## 2. 데이터 결합

In [33]:
# 데이터1 생성
data1 = {
    'id' : ['100','101','102', '103', '104', '106', '108', '110', '111', '113'],
    'city' : ['Soeul', 'Pusan', 'Daegu', 'Gangneung', 'Seoul', 'Seoul', 'Pusan', 'Daegu', 'Gangneung', 'Seoul'],
    'birth_year' : [1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
    'name' : ['Junho', 'Heejin', 'Mijung','Minho', 'Steeve', 'Mina', 'Sumi', 'Minsu', 'Jinhee', 'Daeho']
}

df1 = DataFrame(data1)
df1

,id,city,birth_year,name
0,100,Soeul,1990,Junho
1,101,Pusan,1989,Heejin
2,102,Daegu,1992,Mijung
3,103,Gangneung,1997,Minho
4,104,Seoul,1982,Steeve
5,106,Seoul,1991,Mina
6,108,Pusan,1988,Sumi
7,110,Daegu,1990,Minsu
8,111,Gangneung,1995,Jinhee
9,113,Seoul,1981,Daeho


In [34]:
# 데이터2 생성
data2 = {
    'id' : ['100', '101', '102', '105', '107'],
    'math;' : [50, 43, 33, 76, 98],
    'english' : [90,30,20,50,30],
    'sex' : ['M', 'F', 'F', 'M', 'M'],
    'index_num' : [0,1,2,3,4]
}

df2 = DataFrame(data2)
df2

,id,math;,english,sex,index_num
0,100,50,90,M,0
1,101,43,30,F,1
2,102,33,20,F,2
3,105,76,50,M,3
4,107,98,30,M,4


**결합**

결합의 네 가지 패턴

1. **내부결합(INNSER JOIN)**: 양쪽 데이터에 공통 키 값이 있을 때 결합
2. **외부결합(FULL JOIN)**: 양쪽 모든 데이터를 결합
3. **왼쪽결합(LEFT JOIN)**: 왼쪽 데이터의 키 값을 기준으로 결합
4. **오른쪽결합(RIGHT JOIN)**: 오른쪽 데이터의 키 값을 기준으로 결합

주로 '내부결합'과 '왼쪽결합'을 많이 사용한다. <br>

**내부결합(INNER JOIN)**

merge 메서드는 내부결합이 기본값이다. id를 키 값으로 위 2개의 데이터를 내부 결합하면 아래와 같다. on 파라미터에서 키 값으로 사용할 변수(칼럼)를 지정한다.

In [35]:
# 데이터 결합(내부결합, 키는 자동으로 인식하지만 on으로 명시적으로 지정할 수 있다)
print('결합 테이블')
pd.merge(df1, df2, on = 'id')

결합 테이블


,id,city,birth_year,name,math;,english,sex,index_num
0,100,Soeul,1990,Junho,50,90,M,0
1,101,Pusan,1989,Heejin,43,30,F,1
2,102,Daegu,1992,Mijung,33,20,F,2


**외부결합(OUTER JOIN)**

양쪽 데이터를 모두 결합하는 전결합이다. 전결합 하기 위해 how 파라미터에 **outer**을 지정한다. 결합된 데이터의 특정 변수에 값이 없을 때 NaN으로 표시된다.

In [36]:
# 데이터 결합(전결합)
pd.merge(df1, df2, how = 'outer')

,id,city,birth_year,name,math;,english,sex,index_num
0,100,Soeul,1990.0,Junho,50.0,90.0,M,0.0
1,101,Pusan,1989.0,Heejin,43.0,30.0,F,1.0
2,102,Daegu,1992.0,Mijung,33.0,20.0,F,2.0
3,103,Gangneung,1997.0,Minho,NaN,NaN,NaN,NaN
4,104,Seoul,1982.0,Steeve,NaN,NaN,NaN,NaN
5,106,Seoul,1991.0,Mina,NaN,NaN,NaN,NaN
6,108,Pusan,1988.0,Sumi,NaN,NaN,NaN,NaN
7,110,Daegu,1990.0,Minsu,NaN,NaN,NaN,NaN
8,111,Gangneung,1995.0,Jinhee,NaN,NaN,NaN,NaN
9,113,Seoul,1981.0,Daeho,NaN,NaN,NaN,NaN


left_index 파라미터나 right_on 파라미터로 키를 인덱스로 지정해 결합할 수 있다. 다음 코드는 왼쪽 데이터의 인덱스와 오른쪽 데이터의 index_num 칼럼을 키로 지정한다.

In [37]:
# index를 이용한 결합
pd.merge(df1, df2, left_index = True, right_on = 'index_num')

,id_x,city,birth_year,name,id_y,math;,english,sex,index_num
0,100,Soeul,1990,Junho,100,50,90,M,0
1,101,Pusan,1989,Heejin,101,43,30,F,1
2,102,Daegu,1992,Mijung,102,33,20,F,2
3,103,Gangneung,1997,Minho,105,76,50,M,3
4,104,Seoul,1982,Steeve,107,98,30,M,4


**왼쪽결합(LEFT JOIN)**

왼쪽 결합은 how 파라미터를 left로 지정한다. 다음 예시는 왼쪽 데이터에 맞춰 DataFrame 객체 데이터를 결합한다. 왼쪽 데이터 기준변수(인수)에 대응하는 데이터가 오른쪽 데이터에(두 번째 인수) 없는 경우 해당 칼럼의 값은 NaN이 된다.

In [39]:
# 데이터 결합(left)
pd.merge(df1, df2, how = 'left')

,id,city,birth_year,name,math;,english,sex,index_num
0,100,Soeul,1990,Junho,50.0,90.0,M,0.0
1,101,Pusan,1989,Heejin,43.0,30.0,F,1.0
2,102,Daegu,1992,Mijung,33.0,20.0,F,2.0
3,103,Gangneung,1997,Minho,NaN,NaN,NaN,NaN
4,104,Seoul,1982,Steeve,NaN,NaN,NaN,NaN
5,106,Seoul,1991,Mina,NaN,NaN,NaN,NaN
6,108,Pusan,1988,Sumi,NaN,NaN,NaN,NaN
7,110,Daegu,1990,Minsu,NaN,NaN,NaN,NaN
8,111,Gangneung,1995,Jinhee,NaN,NaN,NaN,NaN
9,113,Seoul,1981,Daeho,NaN,NaN,NaN,NaN


**수직방향 결합**

앞에서는 특정 키를 기준으로 결합했지만,
 concat 메서드를 이용하면 데이터를 수직방향으로 쌓아 올릴 수 있다.

In [40]:
# 데이터3 생성
data3 = {
    'id' : ['117', '118', '119', '120', '125'],
    'city' : ['Ilsan', 'Gunpo', 'Seoul', 'Changwon', 'Jeju'],
    'birth_year' : [1990, 1989, 1992, 1997, 1982],
    'name' : ['Jinhee', 'Yeongho', 'Jongho', 'Yeonghee', 'Hyejin']
}

df3 = DataFrame(data3)
df3

,id,city,birth_year,name
0,117,Ilsan,1990,Jinhee
1,118,Gunpo,1989,Yeongho
2,119,Seoul,1992,Jongho
3,120,Changwon,1997,Yeonghee
4,125,Jeju,1982,Hyejin


In [41]:
# concat 수직방향 결합
concat_data = pd.concat([df1,df3])
concat_data

,id,city,birth_year,name
0,100,Soeul,1990,Junho
1,101,Pusan,1989,Heejin
2,102,Daegu,1992,Mijung
3,103,Gangneung,1997,Minho
4,104,Seoul,1982,Steeve
5,106,Seoul,1991,Mina
6,108,Pusan,1988,Sumi
7,110,Daegu,1990,Minsu
8,111,Gangneung,1995,Jinhee
9,113,Seoul,1981,Daeho




---



## 연습문제

4. 아래 두 가지 데이터를 내부결합 하세요.

In [42]:
# 데이터4 생성
data4 = {
    'id' : ['0', '1', '2', '3', '4', '6', '8', '11', '12', '13'],
    'city' : ['Seoul', 'Pusan', 'Daegu', 'Gangneung', 'Seoul', 'Seoul',
              'Pusan', 'Daeug', 'Gangneung', 'Seoul'],
    'birth_year' : [1990, 1989, 1992, 1997, 1982, 1991, 1988, 1990, 1995, 1981],
    'name' : ['Junho', 'Heejin', 'Mijung', 'Minho', 'Steeve', 'Mina', 'Sumi', 'Minsu', 'Jinhee', 'Daeho']
}

df4 = DataFrame(data4)
df4

,id,city,birth_year,name
0,0,Seoul,1990,Junho
1,1,Pusan,1989,Heejin
2,2,Daegu,1992,Mijung
3,3,Gangneung,1997,Minho
4,4,Seoul,1982,Steeve
5,6,Seoul,1991,Mina
6,8,Pusan,1988,Sumi
7,11,Daeug,1990,Minsu
8,12,Gangneung,1995,Jinhee
9,13,Seoul,1981,Daeho


In [48]:
# 데이터5 생성
data5 = {
    'id' : ['0', '1', '3', '6', '8'],
    'math' : [20, 30, 50, 70, 90],
    'english' : [30,50,50,70,20],
    'sex' : ['M', 'F', 'F', 'M', 'M'],
    'index_num' : [0,1,2,3,4]
}
df5 = DataFrame(data5)
df5

,id,math,english,sex,index_num
0,0,20,30,M,0
1,1,30,50,F,1
2,3,50,50,F,2
3,6,70,70,M,3
4,8,90,20,M,4


In [58]:
# 내부결합
Q4_innerjoin = pd.merge(df4, df5)
Q4_innerjoin

,id,city,birth_year,name,math,english,sex,index_num
0,0,Seoul,1990,Junho,20,30,M,0
1,1,Pusan,1989,Heejin,30,50,F,1
2,3,Gangneung,1997,Minho,50,50,F,2
3,6,Seoul,1991,Mina,70,70,M,3
4,8,Pusan,1988,Sumi,90,20,M,4


5. 문제 4의 데이터를 이용해 df4를 기준으로 df5 테이블을 전결합 하세요

In [52]:
Q5 = pd.merge(df4,df5, how = 'outer')
Q5

,id,city,birth_year,name,math,english,sex,index_num
0,0,Seoul,1990,Junho,20.0,30.0,M,0.0
1,1,Pusan,1989,Heejin,30.0,50.0,F,1.0
2,2,Daegu,1992,Mijung,NaN,NaN,NaN,NaN
3,3,Gangneung,1997,Minho,50.0,50.0,F,2.0
4,4,Seoul,1982,Steeve,NaN,NaN,NaN,NaN
5,6,Seoul,1991,Mina,70.0,70.0,M,3.0
6,8,Pusan,1988,Sumi,90.0,20.0,M,4.0
7,11,Daeug,1990,Minsu,NaN,NaN,NaN,NaN
8,12,Gangneung,1995,Jinhee,NaN,NaN,NaN,NaN
9,13,Seoul,1981,Daeho,NaN,NaN,NaN,NaN


6. 문제4의 데이터를 이용해 df4를 기준으로 아래의 데이터를 수직방향으로 결합하세요.

In [53]:
# 데이터 생성
data6 = {
    'id' : ['70', '80', '90', '120', '150'],
    'city' : ['Ilsan', 'Gunpo', 'Seoul', 'Changwon', 'Jeju'],
    'birth_year' : [1980, 1999, 1995, 1994, 1994],
    'name' : ['Jinhee', 'Yeongho', 'Jongho', 'Yeonghee', 'Hyejin']
}

df6 = DataFrame(data6)
df6

,id,city,birth_year,name
0,70,Ilsan,1980,Jinhee
1,80,Gunpo,1999,Yeongho
2,90,Seoul,1995,Jongho
3,120,Changwon,1994,Yeonghee
4,150,Jeju,1994,Hyejin


In [59]:
# 수직방향 결합: concat
Q6 = pd.concat([df4, df6])
Q6

,id,city,birth_year,name
0,0,Seoul,1990,Junho
1,1,Pusan,1989,Heejin
2,2,Daegu,1992,Mijung
3,3,Gangneung,1997,Minho
4,4,Seoul,1982,Steeve
5,6,Seoul,1991,Mina
6,8,Pusan,1988,Sumi
7,11,Daeug,1990,Minsu
8,12,Gangneung,1995,Jinhee
9,13,Seoul,1981,Daeho


## 3. 데이터 조작과 변환

데이터를 조작 가공하고 변환하는 다양한 방법(피봇 처리, 중복 데이터 처리, 매핑, 구간화 등)

**데이터 피폿**

피봇(pivot)은 데이터의 행을 열로, 열을 행으로 바꾸는 작업이다.

In [60]:
# hier_df 생성
hier_df = DataFrame(
    np.arange(9).reshape((3,3)),
    index = [
        ['a', 'a', 'b'],
        [1,2,2]
    ],
    columns = [
        ['Pusan', 'Seoul', 'Pusan'],
        ['Blue', 'Red', 'Red']
    ]
)

hier_df

Pusan Seoul Pusan
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8